In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [0]:
def area(box):
    return (box[2] - box[0]) * (box[3] - box[1])


def intersection_over_union(boxes):
    assert(len(boxes) == 8)
    boxA = boxes[:4].values
    boxB = boxes[4:].values
    
    boxAArea = area(boxA)
    boxBArea = area(boxB)
    
    if (boxAArea == 0 or boxBArea == 0):
        return 0
        
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    interArea = max(0, xB - xA) * max(0, yB - yA)

    
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

In [0]:
from google.colab import drive
drive.mount('drive')

In [0]:
!unzip -qq "drive/My Drive/crowdsourcing_NIatK4b.zip"

In [0]:
votes = pd.read_csv("drive/My Drive/sna9/train3_data.csv")

In [8]:
votes.head(2)

,userId,itemId,Xmin,Ymin,Xmax,Ymax,Xmin_true,Ymin_true,Xmax_true,Ymax_true,acc,se_Xmin,se_Ymin,se_Xmax,se_Ymax,Y_xmin,Y_ymin,Y_xmax,Y_ymax,L,H,P,S,L/H,D
0,1164,6132,62,656,276,803,47,650,213,812,0.988892,225,36,3969,81,1,1,0,1,214.0,147.0,722.0,31458.0,1.455782,259.624729
1,1399,6132,184,662,235,740,47,650,213,812,0.936663,18769,144,484,5184,0,0,1,0,51.0,78.0,258.0,3978.0,0.653846,93.193347


In [0]:
votes.shape

(5320, 24)

# MACHINE LEARNING.... GO!!!

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier


...датасеты для XGB:

In [9]:
votes.columns

Index(['userId', 'itemId', 'Xmin', 'Ymin', 'Xmax', 'Ymax', 'Xmin_true',
       'Ymin_true', 'Xmax_true', 'Ymax_true', 'acc', 'se_Xmin', 'se_Ymin',
       'se_Xmax', 'se_Ymax', 'Y_xmin', 'Y_ymin', 'Y_xmax', 'Y_ymax', 'L', 'H',
       'P', 'S', 'L/H', 'D'],
      dtype='object')

...по порядку... начну с Xmin

In [0]:
Xmin_X = votes[['userId','Xmin', 'L', 'H', 'P', 'S', 'L/H', 'D']]
Xmin_Y = votes.Y_xmin

In [0]:
X_train, X_test, y_train, y_test = train_test_split(Xmin_X, Xmin_Y, test_size=0.2, random_state=42)

In [15]:
clf = GradientBoostingClassifier().fit(Xmin_X, Xmin_Y )  
clf.score(Xmin_X, Xmin_Y)

0.7046992481203007

... н-да.... негусто. А что со сплитом?

In [16]:
clf = GradientBoostingClassifier().fit(X_train, y_train)  
clf.score(X_test, y_test)

0.6851503759398496

А вот это получше. Имеет смысл прогнать для всех, через функцию

In [0]:
def xgb(A, B):
    X = votes[[A, 'userId','L', 'H', 'P', 'S', 'L/H', 'D']]
    Y = votes[B]
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
    clf = GradientBoostingClassifier().fit(X_train, y_train)  
    print(clf.score(X_test, y_test))

In [18]:
xgb('Ymin', 'Y_ymin') 
xgb('Xmax', 'Y_xmax')
xgb('Ymax', 'Y_ymax')

0.6090225563909775
0.6109022556390977
0.6024436090225563


Ок. Создам предсказания:

In [0]:
def xgb_pred(A, B):
    X = votes[[A, 'userId','L', 'H', 'P', 'S', 'L/H', 'D']]
    Y = votes[B]
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    clf = GradientBoostingClassifier().fit(X, Y)  
    return clf.predict(X) 

In [0]:
votes['pred1'] = xgb_pred('Xmin', 'Y_xmin') 
votes['pred2'] = xgb_pred('Ymin', 'Y_ymin') 
votes['pred3'] = xgb_pred('Xmax', 'Y_xmax')
votes['pred4'] = xgb_pred('Ymax', 'Y_ymax')

In [0]:
votes.head(20)

...теперь как-то надо выкинуть те, в которых нулей больше... одного))
да просто сформирую веса суммированием

In [0]:
votes['weight'] = votes.pred1 + votes.pred2 + votes.pred3 + votes.pred4

In [22]:
votes_good = votes[votes.weight>3]
votes_good.shape

(3401, 30)

In [0]:
answers = pd.read_csv("drive/My Drive/sna9/train_answers.csv")

In [0]:
answers.head()

In [0]:
quorum = votes_good.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].median().reset_index()

In [0]:
quorum.shape

(900, 5)

In [0]:
data = quorum.merge(answers, on=["itemId"])

In [0]:
data["iou"] = data[['Xmin','Ymin', 'Xmax', 'Ymax', 'Xmin_true',\
      'Ymin_true', 'Xmax_true','Ymax_true']].apply(intersection_over_union, axis=1)

In [27]:
data["iou"].mean()

0.5517848696978943

**PREDICTION**

In [0]:
votest = pd.read_csv("drive/My Drive/sna9/test1_data.csv", index_col=0)

In [0]:
votest.head()

,userId,itemId,Xmin,Ymin,Xmax,Ymax,L,H,P,S,L/H,D
0,1581,34804,86,640,242,743,156,103,518,16068,1.514563,186.935818
1,1351,34804,85,655,273,766,188,111,598,20868,1.693694,218.323155
2,161,34804,85,648,268,748,183,100,566,18300,1.830000,208.540164
3,313,5704,32,618,647,1268,615,650,2530,399750,0.946154,894.832387
4,1097,5704,25,620,602,1242,577,622,2398,358894,0.927653,848.417939


In [0]:
def test_pred(A, B, C):
    X = votes[[A, 'userId','L', 'H', 'P', 'S', 'L/H', 'D']]
    Y = votes[B]
    #X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
    clf = GradientBoostingClassifier().fit(X, Y)  
    return clf.predict(C) 

In [0]:
votest['pred1'] = test_pred('Xmin', 'Y_xmin', votest[['Xmin', 'userId','L', 'H', 'P', 'S', 'L/H', 'D']]) 
votest['pred2'] = test_pred('Ymin', 'Y_ymin', votest[['Ymin', 'userId','L', 'H', 'P', 'S', 'L/H', 'D']]) 
votest['pred3'] = test_pred('Xmax', 'Y_xmax', votest[['Xmax', 'userId','L', 'H', 'P', 'S', 'L/H', 'D']])
votest['pred4'] = test_pred('Ymax', 'Y_ymax', votest[['Ymax', 'userId','L', 'H', 'P', 'S', 'L/H', 'D']])

In [31]:
votest['weight'] = votest.pred1 + votest.pred2 + votest.pred3 + votest.pred4
votest_good = votest[votest.weight>3]
votest_good.shape

(2286, 17)

In [32]:
quo_test = votest_good.groupby("itemId")[['Xmin','Ymin', 'Xmax', 'Ymax']].median().reset_index()
quo_test.shape

(583, 5)

In [34]:
best = pd.read_csv("drive/My Drive/sna9/xgb_more3.csv", header=None).rename(columns={0: "itemId", 1: "Xmin", 2:'Ymin', 3:'Xmax', 4:'Ymax'})
best.head(2)

,itemId,Xmin,Ymin,Xmax,Ymax
0,18,144.0,634.0,764.0,1132.0
1,19,28.0,559.0,113.0,727.0


In [0]:
diff = best[best.itemId.isin(set(best.itemId) - set(quo_test.itemId))]

quo_test = pd.concat([quo_test, diff])

In [0]:
quo_test.head()

In [0]:
quo_test.set_index('itemId').sort_index().to_csv("drive/My Drive/sna9/xgb_more33.csv", header=None)

0.5250681

## ПЕРЕДЕЛАЮ КЛАССИФИКАЦИЮ на 1 и 0

In [0]:
""" Найду среднюю медианную ошибку по каждой теме """
mse1 = votes.groupby("itemId")['se_Xmin'].median().reset_index()
mse2 = votes.groupby("itemId")['se_Ymin'].median().reset_index()
mse3 = votes.groupby("itemId")['se_Xmax'].median().reset_index()
mse4 = votes.groupby("itemId")['se_Ymax'].median().reset_index()
#print(mse1,mse2,mse3,mse4) 

In [0]:
mse5 = votes.groupby("itemId")['se_Xmin'].quantile(0.33).reset_index()

In [0]:
mse5.describe()

,itemId,se_Xmin
count,941.000000,941.000000
mean,18241.367694,6088.135430
std,10420.313805,13384.996972
min,5.000000,0.000000
25%,9323.000000,170.320000
50%,18729.000000,1102.980000
75%,27312.000000,5550.330000
max,35962.000000,144481.170000


In [0]:
""" Проставлю оценки """
Y_xmin, Y_ymin, Y_xmax, Y_ymax = [], [], [], []

for i in range(len(votes)):
    for k in range(len(mse1)):
        if votes.itemId[i] == mse1.itemId[k]:
            if votes.se_Xmin[i] < (mse1.se_Xmin[k] * 0.8): Y_xmin.append(1)
            else: Y_xmin.append(0)
Y_xmin  

ValueError: ignored

In [0]:
votes['Y_xmin'] = classes('se_Xmin', mse1, Y_xmin, 0.8)